# Experiments (symbolic + numeric + sweeps + plots)
This notebook implements three reproducible experiments using **SymPy** (analytic derivations) and **NumPy/SciPy** (numerics), includes a parameter sweep harness, plotting, and pytest-compatible unit tests. Outputs (figures/results) are exported to `outputs/` as PNG/SVG/CSV.


In [ ]:
from __future__ import annotations
from pathlib import Path
import numpy as np, pandas as pd
import sympy as sp
from sympy import Eq
from scipy import integrate
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

ROOT=Path('.').resolve(); OUT=ROOT/'outputs'; OUT.mkdir(exist_ok=True)
def savefig(fig,name):
    for ext in ('png','svg'): fig.savefig(OUT/f'{name}.{ext}',bbox_inches='tight',dpi=150)
def savecsv(df,name):
    p=OUT/f'{name}.csv'; df.to_csv(p,index=False); return p
def loadcsv(name): return pd.read_csv(OUT/f'{name}.csv')
np.random.seed(0)
print('OUT_DIR',OUT)


In [ ]:
# --- Experiment A: Gaussian integral I(a)=∫ exp(-a x^2) dx ---
a,x=sp.symbols('a x', positive=True, real=True)
I_sym=sp.integrate(sp.exp(-a*x**2),(x,-sp.oo,sp.oo))
I_closed=sp.simplify(I_sym)
I_num=lambda aval: float(sp.sqrt(sp.pi/aval))
I_quad=lambda aval: integrate.quad(lambda t: np.exp(-aval*t*t),-np.inf,np.inf,limit=200)[0]
sp.pprint(Eq(sp.Symbol('I(a)'),I_closed))

# --- Experiment B: Logistic ODE dN/dt=rN(1-N/K) ---
t=sp.Symbol('t', real=True); r,K,N0=sp.symbols('r K N0', positive=True, real=True)
N=sp.Function('N')
ode=sp.Eq(sp.diff(N(t),t), r*N(t)*(1-N(t)/K))
sol=sp.dsolve(ode, ics={N(0):N0})
N_closed=sp.simplify(sol.rhs)
def logistic_closed(tval,rval,Kval,N0val):
    return float(Kval/(1+((Kval-N0val)/N0val)*np.exp(-rval*tval)))
def logistic_numeric(ts,rval,Kval,N0val):
    f=lambda tt,yy: rval*yy*(1-yy/Kval)
    y0=[N0val]; res=solve_ivp(f,(ts[0],ts[-1]),y0,t_eval=ts,rtol=1e-9,atol=1e-12)
    return res.y[0]
sp.pprint(Eq(sp.Symbol('N(t)'),N_closed))

# --- Experiment C: 1D linear regression y≈mx+b; symbolic normal equation ---
m,b=sp.symbols('m b', real=True)
xi,yi=sp.symbols('x_i y_i', real=True)
X,Y=sp.IndexedBase('X'),sp.IndexedBase('Y'); n=sp.Symbol('n', integer=True, positive=True)
J=sp.summation((Y[i]-(m*X[i]+b))**2,(i,0,n-1))
eqs=[sp.Eq(sp.diff(J,m),0), sp.Eq(sp.diff(J,b),0)]
sol_mb=sp.solve(eqs,[m,b], simplify=True, dict=True)[0]
m_closed=sp.simplify(sol_mb[m]); b_closed=sp.simplify(sol_mb[b])
sp.pprint(Eq(sp.Symbol('m_hat'),m_closed)); sp.pprint(Eq(sp.Symbol('b_hat'),b_closed))
def linreg_numpy(x,y):
    A=np.c_[x,np.ones_like(x)]; (mhat,bhat),_,_,_=np.linalg.lstsq(A,y,rcond=None); return mhat,bhat


In [ ]:
# --- Parameter sweeps + plots + exports ---
rows=[]
# A sweep
for aval in np.logspace(-1,1,13):
    exact=I_num(aval); quad=I_quad(aval); rows.append(dict(exp='gauss',a=aval,exact=exact,numeric=quad,abs_err=abs(quad-exact)))
# B sweep (compare closed form vs solve_ivp at t=5)
ts=np.linspace(0,5,201); Kval=10.0; N0val=0.5
for rval in np.linspace(0.2,2.0,10):
    y_num=logistic_numeric(ts,rval,Kval,N0val)
    y_cl=np.array([logistic_closed(tt,rval,Kval,N0val) for tt in ts])
    rows.append(dict(exp='logistic',r=rval,abs_err=float(np.max(np.abs(y_num-y_cl)))))
# C sweep (MSE vs noise/samples)
rng=np.random.default_rng(0)
mtrue,btrue=2.0,-1.0
for n_samp in (20,50,200):
  x=np.linspace(-1,1,n_samp)
  for sigma in (0.0,0.1,0.3,0.6):
    y=mtrue*x+btrue+rng.normal(0,sigma,size=n_samp)
    mhat,bhat=linreg_numpy(x,y)
    mse=float(np.mean((mhat*x+bhat-(mtrue*x+btrue))**2))
    rows.append(dict(exp='linreg',n=n_samp,sigma=sigma,mse=mse,mhat=mhat,bhat=bhat))
df=pd.DataFrame(rows)
print(df.head())
savecsv(df,'results')

# Plots
fig,ax=plt.subplots(figsize=(4,3))
d=df[df.exp=='gauss'].sort_values('a')
ax.loglog(d.a,d.abs_err,'o-'); ax.set(xlabel='a',ylabel='|quad-exact|',title='Gaussian integral error'); ax.grid(True,which='both')
savefig(fig,'gauss_error')

fig,ax=plt.subplots(figsize=(4,3))
for rval in (0.3,0.8,1.5):
  ax.plot(ts,logistic_numeric(ts,rval,Kval,N0val),label=f'r={rval}')
ax.set(xlabel='t',ylabel='N(t)',title='Logistic ODE (numeric)'); ax.legend(); ax.grid(True)
savefig(fig,'logistic_traj')

fig,ax=plt.subplots(figsize=(4,3))
d=df[df.exp=='linreg']
for n_samp,dd in d.groupby('n'):
  ax.plot(dd.sigma,dd.mse,'o-',label=f'n={n_samp}')
ax.set(xlabel='noise σ',ylabel='MSE',title='Linear regression sensitivity'); ax.legend(); ax.grid(True)
savefig(fig,'linreg_mse')
plt.show()


In [ ]:
# --- Pytest-compatible unit tests (run with: pytest -q) ---
def test_gauss_matches_closed_form():
    for aval in [0.2,1.0,5.0]:
        assert abs(I_quad(aval)-I_num(aval))<5e-10

def test_logistic_numeric_agrees_with_closed_form():
    ts=np.linspace(0,3,101)
    y_num=logistic_numeric(ts,1.1,10.0,0.7)
    y_cl=np.array([logistic_closed(tt,1.1,10.0,0.7) for tt in ts])
    assert float(np.max(np.abs(y_num-y_cl)))<5e-7

def test_linreg_recovery_low_noise():
    rng=np.random.default_rng(1)
    x=np.linspace(-2,2,200); mtrue,btrue=1.5,0.25
    y=mtrue*x+btrue+rng.normal(0,0.05,size=x.size)
    mhat,bhat=linreg_numpy(x,y)
    assert abs(mhat-mtrue)<0.03 and abs(bhat-btrue)<0.03

# quick smoke run inside notebook
if __name__=='__main__':
    test_gauss_matches_closed_form(); test_logistic_numeric_agrees_with_closed_form(); test_linreg_recovery_low_noise();
    print('tests:ok')
